In [ ]:
# import pandas as pd
# from datetime import timedelta

# # ------------------------------------------------------------
# # 1️⃣ 데이터 불러오기
# # ------------------------------------------------------------
# comments = pd.read_excel(
#     r"C:\Users\speec\OneDrive\Desktop\PoC_v2\YouTube_결과\GS리테일_정보유출_comments_GPT_labeled.xlsx"
# )
# posts = pd.read_excel(
#     r"C:\Users\speec\OneDrive\Desktop\PoC_v2\YouTube_결과\GS_posts_GPT_labeled.xlsx"
# )

# # ------------------------------------------------------------
# # 2️⃣ 시간 정리
# # ------------------------------------------------------------
# comments["publishedAt"] = pd.to_datetime(comments["publishedAt"], errors="coerce", utc=True)
# posts["publishedAt"] = pd.to_datetime(posts["publishedAt"], errors="coerce", utc=True)

# # ------------------------------------------------------------
# # 3️⃣ 게시글 기준으로 댓글 병합 (LEFT JOIN)
# # ------------------------------------------------------------
# merged = pd.merge(
#     posts,
#     comments[["videoId", "publishedAt", "gpt_sentiment"]],
#     on="videoId",
#     how="left",
#     suffixes=("_x", "_y")
# )

# # ------------------------------------------------------------
# # 4️⃣ 댓글 작성 시점이 게시글 업로드 후 몇 시간 뒤인지 계산
# # ------------------------------------------------------------
# merged["hours_after"] = (
#     (merged["publishedAt_y"] - merged["publishedAt_x"])
#     .dt.total_seconds() / 3600
# )

# # ------------------------------------------------------------
# # 5️⃣ 사람이 읽기 쉬운 포맷으로 변환 (일, 시, 분)
# # ------------------------------------------------------------
# def format_time_diff(hours):
#     if pd.isna(hours):
#         return "-"
#     days = int(hours // 24)
#     remain_hours = int(hours % 24)
#     minutes = int((hours * 60) % 60)
#     return f"{days}일 {remain_hours}시간 {minutes}분"

# merged["time_diff_display"] = merged["hours_after"].apply(format_time_diff)

# # ------------------------------------------------------------
# # 6️⃣ 컬럼 이름 정리
# # ------------------------------------------------------------
# merged.rename(
#     columns={
#         "publishedAt_x": "video_publishedAt",   # 게시글 업로드 시각
#         "publishedAt_y": "comment_publishedAt"  # 댓글 작성 시각
#     },
#     inplace=True
# )

# # ------------------------------------------------------------
# # 7️⃣ 확인
# # ------------------------------------------------------------
# display(
#     merged[["videoId", "video_publishedAt", "comment_publishedAt", "hours_after", "time_diff_display"]].head(3)
# )


,videoId,video_publishedAt,comment_publishedAt,hours_after,time_diff_display
0,9GukLzTRkkM,2025-02-27 11:45:38+00:00,2025-03-09 07:28:57+00:00,235.721944,9일 19시간 43분
1,9GukLzTRkkM,2025-02-27 11:45:38+00:00,2025-03-08 11:10:03+00:00,215.406944,8일 23시간 24분
2,9GukLzTRkkM,2025-02-27 11:45:38+00:00,2025-03-05 11:22:08+00:00,143.608333,5일 23시간 36분


In [ ]:

# import os
# from datetime import datetime

# # ----------------------------------------
# # 1️⃣ 타임존 제거 (tz-aware → naive)
# # ----------------------------------------
# for col in ["video_publishedAt", "comment_publishedAt"]:
#     if col in merged.columns and pd.api.types.is_datetime64_any_dtype(merged[col]):
#         merged[col] = merged[col].dt.tz_localize(None)

# # ----------------------------------------
# # 2️⃣ 불필요 컬럼 제거 (존재하는 컬럼만 안전하게 drop)
# # ----------------------------------------
# cols_to_drop = [
#     "title", "description", "channelId", "url",
#     "gpt_post_confidence", "comment_publishedAt"
# ]
# existing = [c for c in cols_to_drop if c in merged.columns]
# merged = merged.drop(columns=existing)

# # ----------------------------------------
# # 3️⃣ 출력 경로 자동 생성 (날짜·시간 포함)
# # ----------------------------------------
# timestamp = datetime.now().strftime("%Y%m%d_%H%M")
# output_dir = r"C:\Users\speec\OneDrive\Desktop\PoC_v2\YouTube_결과"
# os.makedirs(output_dir, exist_ok=True)

# output_path = os.path.join(output_dir, f"GS_merged_result_{timestamp}.xlsx")

# # ----------------------------------------
# # 4️⃣ Excel로 저장 (인덱스 제외)
# # ----------------------------------------
# merged.to_excel(output_path, index=False)

# # ----------------------------------------
# # 5️⃣ 로그 출력
# # ----------------------------------------
# print("✅ 병합 파일 저장 완료")
# print(f"📁 경로: {output_path}")
# print(f"📊 저장된 행 개수: {len(merged):,}개")
# print(f"🕓 저장 시각: {timestamp}")


✅ 병합 파일 저장 완료
📁 경로: C:\Users\speec\OneDrive\Desktop\PoC_v2\YouTube_결과\GS_merged_result_20251107_1510.xlsx
📊 저장된 행 개수: 200개
🕓 저장 시각: 20251107_1510


In [ ]:
# df=merged
# # 감정 매핑
# # 감정 매핑
# MAP = {"긍정": 1, "중립": 0, "부정": -1}

# df["post_s"] = df["gpt_post_sentiment"].map(MAP)
# df["cmt_s"]  = df["gpt_sentiment"].map(MAP)

# # videoId 단위로 댓글 평균 감정 계산
# agg = (
#     df.groupby("videoId")
#       .agg(post_s=("post_s", "first"),
#            cmt_s_mean=("cmt_s", "mean"))
#       .reset_index()
# )

# # 절대 괴리만 계산 (0~2 범위)
# agg["abs_sent_gap"] = (agg["post_s"] - agg["cmt_s_mean"]).abs()

# # 결과: 영상별 괴리 정도만 출력
# result = agg[["videoId", "abs_sent_gap"]]

# agg.head(2)




,videoId,post_s,cmt_s_mean,abs_sent_gap
0,0jmI8cbpUY4,-1,-0.722222,0.277778
1,2fCVoYX1dMo,-1,-1.000000,0.000000


In [ ]:
# def calc_alignment(df, t):
#     # t시간 이내의 댓글만 추출
#     sub = df[df["hours_after"] <= t]
#     total = len(sub)
#     if total == 0:
#         return 0  # 댓글이 없을 때는 0개로 처리
    
#     # 영상 감정과 동일한 댓글 개수 계산
#     aligned = (sub["gpt_sentiment"] == sub["gpt_post_sentiment"]).sum()
#     return aligned  # ← 비율이 아니라 개수 반환

# def alignment_features(group):
#     return pd.Series({
#         "aligned_cnt_3h": calc_alignment(group, 3),
#         "aligned_cnt_6h": calc_alignment(group, 6),
#         "aligned_cnt_9h": calc_alignment(group, 9)
#     })

# # 영상(videoId)별로 집계
# align_df = df.groupby("videoId").apply(alignment_features).reset_index()

# align_df.head(2)


C:\Users\speec\AppData\Local\Temp\ipykernel_18160\3392540805.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  align_df = df.groupby("videoId").apply(alignment_features).reset_index()


,videoId,aligned_cnt_3h,aligned_cnt_6h,aligned_cnt_9h
0,0jmI8cbpUY4,7,10,12
1,2fCVoYX1dMo,0,0,0


In [ ]:
# merged_df = pd.merge(align_df, agg, on="videoId", how="inner")
# merged_df.head(2)


,videoId,aligned_cnt_3h,aligned_cnt_6h,aligned_cnt_9h,post_s,cmt_s_mean,abs_sent_gap
0,0jmI8cbpUY4,7,10,12,-1,-0.722222,0.277778
1,2fCVoYX1dMo,0,0,0,-1,-1.000000,0.000000


In [ ]:


# for col in ["video_publishedAt", "comment_publishedAt"]:
#     if col in posts.columns and pd.api.types.is_datetime64_any_dtype(posts[col]):
#         posts[col] = posts[col].dt.tz_localize(None)

# # ----------------------------------------
# # 2️⃣ 불필요 컬럼 제거 (존재하는 컬럼만 안전하게 drop)
# # ----------------------------------------
# cols_to_drop = [
#     "title", "description", "channelId", "url",
#     "gpt_post_confidence", "comment_publishedAt"
# ]
# existing = [c for c in cols_to_drop if c in posts.columns]
# posts = posts.drop(columns=existing)

# posts.head(2)


,videoId,channelTitle,publishedAt,viewCount,likeCount,commentCount,subscriberCount,text_raw,gpt_post_sentiment
0,9GukLzTRkkM,JTBC News,2025-02-27 11:45:38+00:00,13151,228,120,4750000,"주소에 통관번호까지…GS리테일, 158만명 정보 유출 확인 / JTBC 뉴스룸 지난...",부정
1,qcQjIfuls3c,SBS 뉴스,2025-01-06 09:57:34+00:00,6297,68,42,5080000,GS리테일 고객 정보 9만여 건 해킹 피해…3일 만에 알렸다 / SBS / #D리포...,부정


In [ ]:
# # posts, merged_df, align_df 모두 병합 videoId 기준으로

# # 중복 컬럼을 하나만 남기고 정리해서 병합
# final_df = posts.merge(merged_df, on="videoId", how="inner")

# # align_df 에서 이미 병합된 컬럼(merged_df와 동일한 컬럼명)은 제외하고 나머지만 사용
# align_only_cols = [col for col in align_df.columns if col not in final_df.columns or col == "videoId"]

# # 'videoId'는 꼭 들어가야 하므로 포함, 나머지는 중복 아님만 남김
# final_df = final_df.merge(align_df[align_only_cols], on="videoId", how="inner")

# final_df.head(2)


,videoId,channelTitle,publishedAt,viewCount,likeCount,commentCount,subscriberCount,text_raw,gpt_post_sentiment,aligned_cnt_3h,aligned_cnt_6h,aligned_cnt_9h,post_s,cmt_s_mean,abs_sent_gap
0,9GukLzTRkkM,JTBC News,2025-02-27 11:45:38+00:00,13151,228,120,4750000,"주소에 통관번호까지…GS리테일, 158만명 정보 유출 확인 / JTBC 뉴스룸 지난...",부정,7,11,21,-1,-0.820000,0.180000
1,qcQjIfuls3c,SBS 뉴스,2025-01-06 09:57:34+00:00,6297,68,42,5080000,GS리테일 고객 정보 9만여 건 해킹 피해…3일 만에 알렸다 / SBS / #D리포...,부정,18,21,22,-1,-0.783784,0.216216


In [ ]:
# # ✅ 1️⃣ 시간대(tz) 제거 후 저장
# final_df["publishedAt"] = final_df["publishedAt"].dt.tz_localize(None)

# # ✅ 2️⃣ 엑셀로 저장
# final_df.to_excel(
#     r"C:\Users\speec\OneDrive\Desktop\PoC_v2\final_GS_result.xlsx",
#     index=False,
#     engine="openpyxl"
# )

# print("✅ 엑셀 저장 완료!")


✅ 엑셀 저장 완료!


In [3]:
import pandas as pd
import os

# ============================================================
# ✅ 1. 기본 설정
# ============================================================
BASE_PATH = r"C:\Users\speec\OneDrive\Desktop\PoC_v2\YouTube_결과"
ISSUES = ["GS", "KT", "롯데", "SKT"]

# ============================================================
# ✅ 2. 함수 정의
# ============================================================

def load_data(issue):
    posts = pd.read_excel(os.path.join(BASE_PATH, f"{issue}_posts_GPT_labeled.xlsx"))
    comments = pd.read_excel(os.path.join(BASE_PATH, f"{issue}_댓글감정분석_GPT_labeled.xlsx"))
    posts["publishedAt"] = pd.to_datetime(posts["publishedAt"], errors="coerce", utc=True)
    comments["publishedAt"] = pd.to_datetime(comments["publishedAt"], errors="coerce", utc=True)
    return posts, comments


def merge_post_comment(posts, comments):
    merged = pd.merge(
        posts,
        comments[["videoId", "publishedAt", "gpt_sentiment"]],
        on="videoId",
        how="left",
        suffixes=("_post", "_comment")
    )
    merged["hours_after"] = (
        (merged["publishedAt_comment"] - merged["publishedAt_post"]).dt.total_seconds() / 3600
    )
    return merged


def calc_alignment(merged):
    def to_score(x):
        if isinstance(x, str):
            if "부정" in x:
                return -1
            elif "긍정" in x:
                return 1
        return 0

    merged["post_s"] = merged["gpt_post_sentiment"].apply(to_score)
    merged["cmt_s"] = merged["gpt_sentiment"].apply(to_score)

    grouped = (
        merged.groupby("videoId")
        .agg(
            channelTitle=("channelTitle", "first"),
            publishedAt=("publishedAt_post", "first"),
            viewCount=("viewCount", "first"),
            likeCount=("likeCount", "first"),
            commentCount=("commentCount", "first"),
            subscriberCount=("subscriberCount", "first"),
            text_raw=("text_raw", "first"),
            gpt_post_sentiment=("gpt_post_sentiment", "first"),
            post_s=("post_s", "first"),
            cmt_s_mean=("cmt_s", "mean")
        )
        .reset_index()
    )

    grouped["abs_sent_gap"] = (grouped["post_s"] - grouped["cmt_s_mean"]).abs().round(2)

    for h in [3, 6, 9]:
        cond = (merged["hours_after"] <= h) & (merged["post_s"] == merged["cmt_s"])
        aligned = merged[cond].groupby("videoId").size()
        grouped[f"aligned_cnt_{h}h"] = grouped["videoId"].map(aligned).fillna(0).astype(int)

    return grouped


# ============================================================
# ✅ 3. 실행 루프
# ============================================================
for issue in ISSUES:
    print(f"\n🚀 {issue} 처리 중...")

    posts, comments = load_data(issue)
    merged = merge_post_comment(posts, comments)
    result = calc_alignment(merged)

    # Excel 저장 전 timezone 제거
    for col in merged.select_dtypes(include=["datetimetz"]).columns:
        merged[col] = merged[col].dt.tz_localize(None)   # ← 여기 들여쓰기 필수!

    for col in result.select_dtypes(include=["datetimetz"]).columns:
        result[col] = result[col].dt.tz_localize(None)   # ← 여기도 들여쓰기 필수!

    # 저장
    merged.to_excel(os.path.join(BASE_PATH, f"{issue}_merged_timegap.xlsx"), index=False, engine="openpyxl")
    result.to_excel(os.path.join(BASE_PATH, f"{issue}_final_post_level.xlsx"), index=False, engine="openpyxl")

    print(f"✅ {issue} 완료: 댓글 {len(merged):,}건 → 게시글 {len(result):,}건")

print("\n🎯 모든 이슈 처리 완료!")



🚀 GS 처리 중...
✅ GS 완료: 댓글 200건 → 게시글 22건

🚀 KT 처리 중...
✅ KT 완료: 댓글 6,670건 → 게시글 349건

🚀 롯데 처리 중...
✅ 롯데 완료: 댓글 5,611건 → 게시글 500건

🚀 SKT 처리 중...
✅ SKT 완료: 댓글 7,658건 → 게시글 494건

🎯 모든 이슈 처리 완료!


In [21]:
import pandas as pd
import os

# ------------------------------------------------------------
# 1️⃣ 경로 및 이슈 목록
# ------------------------------------------------------------
BASE_PATH = r"C:\Users\speec\OneDrive\Desktop\PoC_v2\공유폴더(딥테크)"
ISSUES = ["GS", "KT", "롯데", "SKT"]

# ------------------------------------------------------------
# 2️⃣ 댓글 수를 게시글에 추가하는 함수 + 전체 댓글 개수 출력
# ------------------------------------------------------------
def add_comment_count_to_posts(issue):
    """댓글 수를 게시글 파일에 추가한 뒤 저장하며, 전체 댓글 수도 출력"""
    # 파일 불러오기
    post_path = os.path.join(BASE_PATH, f"{issue}_뉴스감정분석_GPT_labeled.xlsx")
    cmt_path = os.path.join(BASE_PATH, f"{issue}_댓글감정분석_GPT_labeled.xlsx")

    posts = pd.read_excel(post_path)
    comments = pd.read_excel(cmt_path)

    # 병합 키 통일 (URL 기준)
    posts["videoId"] = posts["url"]
    comments["videoId"] = comments["url"]

    # ✅ 댓글 수 계산 (videoId 기준)
    comment_counts = comments.groupby("videoId").size().rename("comment_cnt").reset_index()

    # ✅ 게시글에 댓글수 추가 (없으면 0)
    posts = posts.merge(comment_counts, on="videoId", how="left").fillna({"comment_cnt": 0})
    posts["comment_cnt"] = posts["comment_cnt"].astype(int)

    # ✅ 전체 댓글 수 카운트
    total_comments = comments.shape[0]

    # ✅ 결과 저장
    output_path = os.path.join(BASE_PATH, f"{issue}_posts_with_comment_cnt.xlsx")
    posts.to_excel(output_path, index=False, engine="openpyxl")

    print(f"✅ {issue} 완료 — 게시글 {len(posts)}건, 댓글수 추가 후 저장됨")
    print(f"📂 저장 위치: {output_path}")
    print(f"💬 {issue} 이슈 댓글 총 {total_comments:,}개가 추가 및 집계되었습니다.\n")

# ------------------------------------------------------------
# 3️⃣ 실행
# ------------------------------------------------------------
for issue in ISSUES:
    add_comment_count_to_posts(issue)

print("\n🎯 모든 이슈 게시글에 댓글수가 추가되었습니다!")


✅ GS 완료 — 게시글 186건, 댓글수 추가 후 저장됨
📂 저장 위치: C:\Users\speec\OneDrive\Desktop\PoC_v2\공유폴더(딥테크)\GS_posts_with_comment_cnt.xlsx
💬 GS 이슈 댓글 총 593개가 추가 및 집계되었습니다.

✅ KT 완료 — 게시글 436건, 댓글수 추가 후 저장됨
📂 저장 위치: C:\Users\speec\OneDrive\Desktop\PoC_v2\공유폴더(딥테크)\KT_posts_with_comment_cnt.xlsx
💬 KT 이슈 댓글 총 2,248개가 추가 및 집계되었습니다.

✅ 롯데 완료 — 게시글 340건, 댓글수 추가 후 저장됨
📂 저장 위치: C:\Users\speec\OneDrive\Desktop\PoC_v2\공유폴더(딥테크)\롯데_posts_with_comment_cnt.xlsx
💬 롯데 이슈 댓글 총 2,287개가 추가 및 집계되었습니다.

✅ SKT 완료 — 게시글 881건, 댓글수 추가 후 저장됨
📂 저장 위치: C:\Users\speec\OneDrive\Desktop\PoC_v2\공유폴더(딥테크)\SKT_posts_with_comment_cnt.xlsx
💬 SKT 이슈 댓글 총 15,388개가 추가 및 집계되었습니다.


🎯 모든 이슈 게시글에 댓글수가 추가되었습니다!


In [ ]:
여기부터 하기   유튭에서 네이버 형식에 맞게 고치기 

In [9]:
posts = pd.read_excel(os.path.join(BASE_PATH, f"{'GS'}_posts_with_comment_cnt.xlsx"))
comments = pd.read_excel(os.path.join(BASE_PATH, f"{'GS'}_댓글감정분석_GPT_labeled.xlsx"))

posts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   키워드             186 non-null    object 
 1   언론사             186 non-null    object 
 2   게시일             186 non-null    object 
 3   뉴스제목            186 non-null    object 
 4   url             186 non-null    object 
 5   gpt_sentiment   186 non-null    object 
 6   gpt_confidence  186 non-null    float64
 7   videoId         186 non-null    object 
 8   comment_cnt     186 non-null    int64  
dtypes: float64(1), int64(1), object(7)
memory usage: 13.2+ KB


In [10]:
comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 593 entries, 0 to 592
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   channel         593 non-null    object 
 1   url             593 non-null    object 
 2   title           0 non-null      float64
 3   post_date       593 non-null    object 
 4   comment         593 non-null    object 
 5   gpt_sentiment   593 non-null    object 
 6   gpt_confidence  593 non-null    float64
dtypes: float64(2), object(5)
memory usage: 32.6+ KB


In [13]:
import pandas as pd
import os

# ============================================================
# ✅ 1. 기본 설정
# ============================================================
BASE_PATH = r"C:\Users\user\Desktop\PoC_v2\공유폴더(딥테크)"
ISSUES = ["GS", "KT", "롯데", "SKT"]

# ============================================================
# ✅ 2. 함수 정의
# ============================================================

def load_data(issue):
    """posts / comments 데이터 로드 + 날짜 처리"""
    posts = pd.read_excel(os.path.join(BASE_PATH, f"{issue}_posts_with_comment_cnt.xlsx"))
    comments = pd.read_excel(os.path.join(BASE_PATH, f"{issue}_댓글감정분석_GPT_labeled.xlsx"))
    
    posts["publishedAt"] = pd.to_datetime(posts["게시일"], errors="coerce", utc=True)
    comments["publishedAt"] = pd.to_datetime(comments["post_date"], errors="coerce", utc=True)
    
    return posts, comments


def merge_post_comment(posts, comments):
    """게시글과 댓글 데이터 병합 + 시간차 계산"""
    merged = pd.merge(
        posts,
        comments[["url", "publishedAt", "gpt_sentiment"]],
        on="url",
        how="left",
        suffixes=("_post", "_comment")
    )

    merged["hours_after"] = (
        (merged["publishedAt_comment"] - merged["publishedAt_post"]).dt.total_seconds() / 3600
    )
    return merged


def calc_alignment(merged):
    """게시글-댓글 감정 정렬도 계산"""
    def to_score(x):
        if isinstance(x, str):
            if "부정" in x:
                return -1
            elif "긍정" in x:
                return 1
        return 0

    merged["post_s"] = merged["gpt_sentiment_post"].apply(to_score)
    merged["cmt_s"] = merged["gpt_sentiment_comment"].apply(to_score)

    # videoId 컬럼이 없으므로 url 기준으로 그룹화
    grouped = (
        merged.groupby("url")
        .agg(
            키워드=("키워드", "first"),
            언론사=("언론사", "first"),
            뉴스제목=("뉴스제목", "first"),
            게시일=("게시일", "first"),
            post_sentiment=("gpt_sentiment_post", "first"),
            post_s=("post_s", "first"),
            cmt_s_mean=("cmt_s", "mean"),
            comment_cnt=("comment_cnt", "first"),
        )
        .reset_index()
    )

    # 감정 차이
    grouped["abs_sent_gap"] = (grouped["post_s"] - grouped["cmt_s_mean"]).abs().round(2)

    # 3/6/9시간 내 동일감정 댓글 수 카운트
    for h in [3, 6, 9]:
        cond = (merged["hours_after"] <= h) & (merged["post_s"] == merged["cmt_s"])
        aligned = merged[cond].groupby("url").size()
        grouped[f"aligned_cnt_{h}h"] = grouped["url"].map(aligned).fillna(0).astype(int)

    return grouped


# ============================================================
# ✅ 3. 실행 루프
# ============================================================
for issue in ISSUES:
    print(f"\n🚀 {issue} 처리 중...")

    posts, comments = load_data(issue)
    merged = merge_post_comment(posts, comments)
    result = calc_alignment(merged)

    # Excel 저장 전 timezone 제거
    for col in merged.select_dtypes(include=["datetimetz"]).columns:
        merged[col] = merged[col].dt.tz_localize(None)
    for col in result.select_dtypes(include=["datetimetz"]).columns:
        result[col] = result[col].dt.tz_localize(None)

    # 저장
    merged.to_excel(os.path.join(BASE_PATH, f"{issue}_merged_timegap.xlsx"), index=False, engine="openpyxl")
    result.to_excel(os.path.join(BASE_PATH, f"{issue}_final_post_level.xlsx"), index=False, engine="openpyxl")

    
print("\n🎯 모든 이슈 처리 완료!")



🚀 GS 처리 중...

🚀 KT 처리 중...

🚀 롯데 처리 중...

🚀 SKT 처리 중...

🎯 모든 이슈 처리 완료!


In [9]:
import os
import pandas as pd

# ------------------------------------------------------------
# 1️⃣ 경로 설정
# ------------------------------------------------------------
base_path = r"C:\Users\user\Desktop\PoC_v2\공유폴더(딥테크)"
post_path = os.path.join(base_path, "final_post_1")
script_path = os.path.join(base_path, "script")

# ------------------------------------------------------------
# 2️⃣ 이슈별 매핑
# ------------------------------------------------------------
issue_map = {
    "gs": "GS_final_post_level.xlsx",
    "kt": "KT_final_post_level.xlsx",
    "lotte": "롯데_final_post_level.xlsx",
    "skt": "SKT_final_post_level.xlsx"
}

# ------------------------------------------------------------
# 3️⃣ 병합 실행 (url 기준으로 script의 context만 붙이기)
# ------------------------------------------------------------
for issue, post_filename in issue_map.items():
    post_file = os.path.join(post_path, post_filename)
    script_file = os.path.join(script_path, f"{issue}_script.csv")

    if not os.path.exists(post_file):
        print(f"⚠️ {post_filename} 없음 — 건너뜀")
        continue
    if not os.path.exists(script_file):
        print(f"⚠️ {issue}_script.csv 없음 — 건너뜀")
        continue

    # 엑셀, CSV 불러오기
    post_df = pd.read_excel(post_file)
    script_df = pd.read_csv(script_file)

    # --------------------------------------------------------
    # 4️⃣ URL 전처리 (공백, 대소문자, http/https 정규화)
    # --------------------------------------------------------
    for df in [post_df, script_df]:
        df['url'] = df['url'].astype(str).str.strip().str.lower()
        df['url'] = df['url'].str.replace('https://', 'http://', regex=False)
        df['url'] = df['url'].str.rstrip('/')

    # --------------------------------------------------------
    # 5️⃣ 병합 (context만 추가)
    # --------------------------------------------------------
    if 'context' not in script_df.columns:
        print(f"⚠️ {issue}: script에 'context' 컬럼 없음 — 빈값 추가")
        script_df['context'] = ""

    # 기존 post_df에 context 컬럼이 있다면 제거 후 갱신
    if 'context' in post_df.columns:
        post_df = post_df.drop(columns=['context'])

    # url 기준 병합
    merged = post_df.merge(
        script_df[['url', 'context']],
        on='url',
        how='left'
    )

    # --------------------------------------------------------
    # 6️⃣ 덮어쓰기 저장 (같은 파일명)
    # --------------------------------------------------------
    merged.to_excel(post_file, index=False, engine='openpyxl')
    print(f"✅ {post_filename} 업데이트 완료 — context 정확히 병합됨 ({len(merged)}행)")

print("\n🎯 모든 이슈 파일에 script의 context 컬럼이 URL 기준으로 정상 병합되었습니다.")


⚠️ gs: script에 'context' 컬럼 없음 — 빈값 추가
✅ GS_final_post_level.xlsx 업데이트 완료 — context 정확히 병합됨 (186행)
⚠️ kt: script에 'context' 컬럼 없음 — 빈값 추가
✅ KT_final_post_level.xlsx 업데이트 완료 — context 정확히 병합됨 (436행)
⚠️ lotte: script에 'context' 컬럼 없음 — 빈값 추가
✅ 롯데_final_post_level.xlsx 업데이트 완료 — context 정확히 병합됨 (340행)
⚠️ skt: script에 'context' 컬럼 없음 — 빈값 추가
✅ SKT_final_post_level.xlsx 업데이트 완료 — context 정확히 병합됨 (881행)

🎯 모든 이슈 파일에 script의 context 컬럼이 URL 기준으로 정상 병합되었습니다.


In [14]:
import pandas as pd
import os

# ============================================================
# ✅ 1. 경로 설정
# ============================================================
YOUTUBE_PATH = r"C:\Users\user\Desktop\PoC_v2\YouTube_결과\final_post"
NAVER_PATH = r"C:\Users\user\Desktop\PoC_v2\공유폴더(딥테크)\final_post_1"
ISSUES = ["GS", "KT", "롯데", "SKT"]

# ============================================================
# ✅ 2. 각 파일 컬럼명 출력
# ============================================================
for issue in ISSUES:
    yt_path = os.path.join(YOUTUBE_PATH, f"{issue}_final_post_level.xlsx")
    nv_path = os.path.join(NAVER_PATH, f"{issue}_final_post_level.xlsx")

    print(f"\n============================")
    print(f"📂 {issue} 컬럼 구조 확인")
    print("============================")

    if os.path.exists(yt_path):
        yt_cols = pd.read_excel(yt_path, nrows=1).columns.tolist()
        print(f"🟦 유튜브 ({issue}): {yt_path}")
        print(f"→ {yt_cols}\n")
    else:
        print(f"⚠️ 유튜브 파일 없음: {yt_path}")

    if os.path.exists(nv_path):
        nv_cols = pd.read_excel(nv_path, nrows=1).columns.tolist()
        print(f"🟩 네이버 ({issue}): {nv_path}")
        print(f"→ {nv_cols}\n")
    else:
        print(f"⚠️ 네이버 파일 없음: {nv_path}")



📂 GS 컬럼 구조 확인
🟦 유튜브 (GS): C:\Users\user\Desktop\PoC_v2\YouTube_결과\final_post\GS_final_post_level.xlsx
→ ['videoId', 'channelTitle', 'publishedAt', 'viewCount', 'likeCount', 'commentCount', 'subscriberCount', 'text_raw', 'gpt_post_sentiment', 'post_s', 'cmt_s_mean', 'abs_sent_gap', 'aligned_cnt_3h', 'aligned_cnt_6h', 'aligned_cnt_9h']

🟩 네이버 (GS): C:\Users\user\Desktop\PoC_v2\공유폴더(딥테크)\final_post_1\GS_final_post_level.xlsx
→ ['url', '키워드', '언론사', '뉴스제목', '게시일', 'post_sentiment', 'post_s', 'cmt_s_mean', 'comment_cnt', 'abs_sent_gap', 'aligned_cnt_3h', 'aligned_cnt_6h', 'aligned_cnt_9h']


📂 KT 컬럼 구조 확인
🟦 유튜브 (KT): C:\Users\user\Desktop\PoC_v2\YouTube_결과\final_post\KT_final_post_level.xlsx
→ ['videoId', 'channelTitle', 'publishedAt', 'viewCount', 'likeCount', 'commentCount', 'subscriberCount', 'text_raw', 'gpt_post_sentiment', 'post_s', 'cmt_s_mean', 'abs_sent_gap', 'aligned_cnt_3h', 'aligned_cnt_6h', 'aligned_cnt_9h']

🟩 네이버 (KT): C:\Users\user\Desktop\PoC_v2\공유폴더(딥테크)\final_post_1\KT_f

In [5]:
import pandas as pd
import os

# ============================================================
# ✅ 1. 기본 설정
# ============================================================
YOUTUBE_PATH = r"C:\Users\user\Desktop\PoC_v2\YouTube_결과\final_post"
NAVER_PATH = r"C:\Users\user\Desktop\PoC_v2\공유폴더(딥테크)\final_post_1"
OUTPUT_PATH = r"C:\Users\user\Desktop\PoC_v2\통합결과_날짜별"

ISSUES = ["GS", "KT", "롯데", "SKT"]

# ============================================================
# ✅ 2. 날짜 통일 함수
# ============================================================

def normalize_date_column(df):
    """YouTube: publishedAt → date / Naver: 게시일 → date"""
    if "publishedAt" in df.columns:
        df["date"] = pd.to_datetime(df["publishedAt"], errors="coerce").dt.date
    elif "게시일" in df.columns:
        df["date"] = pd.to_datetime(df["게시일"], errors="coerce").dt.date
    else:
        df["date"] = pd.NaT
    return df


# ============================================================
# ✅ 3. 플랫폼별 로드 함수
# ============================================================

def load_youtube(issue):
    path = os.path.join(YOUTUBE_PATH, f"{issue}_final_post_level.xlsx")
    if not os.path.exists(path):
        print(f"⚠️ 유튜브 파일 없음: {path}")
        return pd.DataFrame()

    df = pd.read_excel(path)
    df = normalize_date_column(df)

    df = df.rename(
        columns={
            "videoId": "content_id",
            "gpt_post_sentiment": "post_sentiment",
            "text_raw": "title_or_text"
        }
    )
    df["channel"] = "유튜브"

    keep_cols = [
        "content_id", "channel", "channelTitle", "title_or_text",
        "post_sentiment", "post_s", "cmt_s_mean", "abs_sent_gap",
        "aligned_cnt_3h", "aligned_cnt_6h", "aligned_cnt_9h",
        "viewCount", "likeCount", "commentCount", "subscriberCount", "date"
    ]
    df = df[[c for c in keep_cols if c in df.columns]]

    # 유튜브엔 comment_cnt 없으므로 commentCount → comment_cnt로 통합
    if "commentCount" in df.columns:
        df = df.rename(columns={"commentCount": "comment_cnt"})

    return df


def load_naver(issue):
    path = os.path.join(NAVER_PATH, f"{issue}_final_post_level.xlsx")
    if not os.path.exists(path):
        print(f"⚠️ 네이버 파일 없음: {path}")
        return pd.DataFrame()

    df = pd.read_excel(path)
    df = normalize_date_column(df)
    df = df.drop(columns=["키워드"], errors="ignore")

    df = df.rename(
        columns={
            "url": "content_id",
            "뉴스제목": "title_or_text"
        }
    )
    df["channel"] = "네이버"

    keep_cols = [
        "content_id", "channel", "언론사", "title_or_text",
        "post_sentiment", "post_s", "cmt_s_mean", "abs_sent_gap",
        "aligned_cnt_3h", "aligned_cnt_6h", "aligned_cnt_9h",
        "comment_cnt", "date","content"
    ]
    df = df[[c for c in keep_cols if c in df.columns]]
    return df


# ============================================================
# ✅ 4. 병합 및 저장
# ============================================================

os.makedirs(OUTPUT_PATH, exist_ok=True)

for issue in ISSUES:
    yt_df = load_youtube(issue)
    nv_df = load_naver(issue)

    combined = pd.concat([yt_df, nv_df], ignore_index=True)

    # ✅ comment_cnt NaN → 0
    if "comment_cnt" in combined.columns:
        combined["comment_cnt"] = combined["comment_cnt"].fillna(0).astype(int)

    # ✅ 날짜 기준 정렬
    combined = combined.sort_values(by="date")

    # ✅ date 컬럼을 가장 왼쪽으로 이동
    cols = ["date"] + [c for c in combined.columns if c != "date"]
    combined = combined[cols]

    # 저장
    output_file = os.path.join(OUTPUT_PATH, f"{issue}_통합_final.xlsx")
    combined.to_excel(output_file, index=False, engine="openpyxl")

    print(f"✅ {issue} 통합 완료 ({len(combined)}행 저장) → {output_file}")

print("\n🎯 모든 이슈 통합 완료 (date 맨 왼쪽 + comment_cnt 결측치 0 처리 완료)")


✅ GS 통합 완료 (208행 저장) → C:\Users\user\Desktop\PoC_v2\통합결과_날짜별\GS_통합_final.xlsx
✅ KT 통합 완료 (785행 저장) → C:\Users\user\Desktop\PoC_v2\통합결과_날짜별\KT_통합_final.xlsx
✅ 롯데 통합 완료 (840행 저장) → C:\Users\user\Desktop\PoC_v2\통합결과_날짜별\롯데_통합_final.xlsx
✅ SKT 통합 완료 (1375행 저장) → C:\Users\user\Desktop\PoC_v2\통합결과_날짜별\SKT_통합_final.xlsx

🎯 모든 이슈 통합 완료 (date 맨 왼쪽 + comment_cnt 결측치 0 처리 완료)


In [22]:
# import pandas as pd
# import os

# # ============================================================
# # ✅ 1. 기본 설정
# # ============================================================
# YOUTUBE_PATH = r"C:\Users\user\Desktop\PoC_v2\YouTube_결과\final_post"
# NAVER_PATH = r"C:\Users\user\Desktop\PoC_v2\공유폴더(딥테크)\final_post_1"
# OUTPUT_PATH = r"C:\Users\user\Desktop\PoC_v2\통합결과_날짜별"

# ISSUES = ["GS", "KT", "롯데", "SKT"]

# # ============================================================
# # ✅ 2. 날짜 통일 함수
# # ============================================================

# def normalize_date_column(df):
#     """YouTube: publishedAt → date / Naver: 게시일 → date"""
#     if "publishedAt" in df.columns:
#         df["date"] = pd.to_datetime(df["publishedAt"], errors="coerce").dt.date
#     elif "게시일" in df.columns:
#         df["date"] = pd.to_datetime(df["게시일"], errors="coerce").dt.date
#     else:
#         df["date"] = pd.NaT
#     return df


# # ============================================================
# # ✅ 3. 플랫폼별 로드 함수
# # ============================================================

# def load_youtube(issue):
#     path = os.path.join(YOUTUBE_PATH, f"{issue}_final_post_level.xlsx")
#     if not os.path.exists(path):
#         print(f"⚠️ 유튜브 파일 없음: {path}")
#         return pd.DataFrame()

#     df = pd.read_excel(path)
#     df = normalize_date_column(df)

#     df = df.rename(
#         columns={
#             "videoId": "content_id",
#             "gpt_post_sentiment": "post_sentiment",
#             "text_raw": "title_or_text"
#         }
#     )
#     df["channel"] = "유튜브"

#     keep_cols = [
#         "content_id", "channel", "channelTitle", "title_or_text",
#         "post_sentiment", "post_s", "cmt_s_mean", "abs_sent_gap",
#         "aligned_cnt_3h", "aligned_cnt_6h", "aligned_cnt_9h",
#         "viewCount", "likeCount", "commentCount", "subscriberCount", "date"
#     ]
#     df = df[[c for c in keep_cols if c in df.columns]]

#     # 유튜브엔 comment_cnt 없으므로 commentCount → comment_cnt로 통합
#     if "commentCount" in df.columns:
#         df = df.rename(columns={"commentCount": "comment_cnt"})

#     return df


# def load_naver(issue):
#     path = os.path.join(NAVER_PATH, f"{issue}_final_post_level.xlsx")
#     if not os.path.exists(path):
#         print(f"⚠️ 네이버 파일 없음: {path}")
#         return pd.DataFrame()

#     df = pd.read_excel(path)
#     df = normalize_date_column(df)
#     df = df.drop(columns=["키워드"], errors="ignore")

#     df = df.rename(
#         columns={
#             "url": "content_id",
#             "뉴스제목": "title_or_text"
#         }
#     )
#     df["channel"] = "네이버"

#     keep_cols = [
#         "content_id", "channel", "언론사", "title_or_text",
#         "post_sentiment", "post_s", "cmt_s_mean", "abs_sent_gap",
#         "aligned_cnt_3h", "aligned_cnt_6h", "aligned_cnt_9h",
#         "comment_cnt", "date","content"
#     ]
#     df = df[[c for c in keep_cols if c in df.columns]]
#     return df


# # ============================================================
# # ✅ 4. 병합 및 저장
# # ============================================================

# os.makedirs(OUTPUT_PATH, exist_ok=True)

# for issue in ISSUES:
#     yt_df = load_youtube(issue)
#     nv_df = load_naver(issue)

#     combined = pd.concat([yt_df, nv_df], ignore_index=True)

#     # ✅ comment_cnt NaN → 0
#     if "comment_cnt" in combined.columns:
#         combined["comment_cnt"] = combined["comment_cnt"].fillna(0).astype(int)

#     # ✅ 날짜 기준 정렬
#     combined = combined.sort_values(by="date")

#     # ✅ date 컬럼을 가장 왼쪽으로 이동
#     cols = ["date"] + [c for c in combined.columns if c != "date"]
#     combined = combined[cols]

#     # 저장
#     output_file = os.path.join(OUTPUT_PATH, f"{issue}_통합_final.xlsx")
#     combined.to_excel(output_file, index=False, engine="openpyxl")

#     print(f"✅ {issue} 통합 완료 ({len(combined)}행 저장) → {output_file}")

# print("\n🎯 모든 이슈 통합 완료 (date 맨 왼쪽 + comment_cnt 결측치 0 처리 완료)")


✅ GS 통합 완료 (208행 저장) → C:\Users\user\Desktop\PoC_v2\통합결과_날짜별\GS_통합_final.xlsx
✅ KT 통합 완료 (785행 저장) → C:\Users\user\Desktop\PoC_v2\통합결과_날짜별\KT_통합_final.xlsx
✅ 롯데 통합 완료 (840행 저장) → C:\Users\user\Desktop\PoC_v2\통합결과_날짜별\롯데_통합_final.xlsx
✅ SKT 통합 완료 (1375행 저장) → C:\Users\user\Desktop\PoC_v2\통합결과_날짜별\SKT_통합_final.xlsx

🎯 모든 이슈 통합 완료 (date 맨 왼쪽 + comment_cnt 결측치 0 처리 완료)


In [6]:
import pandas as pd
import os

# ============================================================
# ✅ 1. 기본 설정
# ============================================================
YOUTUBE_PATH = r"C:\Users\user\Desktop\PoC_v2\YouTube_결과\final_post"
NAVER_PATH = r"C:\Users\user\Desktop\PoC_v2\공유폴더(딥테크)\final_post_1"
OUTPUT_PATH = r"C:\Users\user\Desktop\PoC_v2\통합결과_날짜시간"

ISSUES = ["GS", "KT", "롯데", "SKT"]

# ============================================================
# ✅ 2. 날짜 + 시간 통일 함수 (KST 유지)
# ============================================================

def normalize_datetime_column(df):
    """YouTube: publishedAt / Naver: 게시일 → datetime (KST) 변환, 시간 포함"""
    if "publishedAt" in df.columns:
        df["datetime"] = (
            pd.to_datetime(df["publishedAt"], errors="coerce", utc=True)
            .dt.tz_convert("Asia/Seoul")
            .dt.tz_localize(None)
        )
    elif "게시일" in df.columns:
        df["datetime"] = (
            pd.to_datetime(df["게시일"], errors="coerce", utc=True)
            .dt.tz_convert("Asia/Seoul")
            .dt.tz_localize(None)
        )
    else:
        df["datetime"] = pd.NaT
    return df


# ============================================================
# ✅ 3. 플랫폼별 로드 함수
# ============================================================

def load_youtube(issue):
    path = os.path.join(YOUTUBE_PATH, f"{issue}_final_post_level.xlsx")
    if not os.path.exists(path):
        print(f"⚠️ 유튜브 파일 없음: {path}")
        return pd.DataFrame()

    df = pd.read_excel(path)
    df = normalize_datetime_column(df)

    df = df.rename(
        columns={
            "videoId": "content_id",
            "gpt_post_sentiment": "post_sentiment",
            "text_raw": "title_or_text"
        }
    )
    df["channel"] = "유튜브"

    keep_cols = [
        "content_id", "channel", "channelTitle", "title_or_text",
        "post_sentiment", "post_s", "cmt_s_mean", "abs_sent_gap",
        "aligned_cnt_3h", "aligned_cnt_6h", "aligned_cnt_9h",
        "viewCount", "likeCount", "commentCount", "subscriberCount", "datetime"
    ]
    df = df[[c for c in keep_cols if c in df.columns]]

    # 유튜브 commentCount → comment_cnt 통일
    if "commentCount" in df.columns:
        df = df.rename(columns={"commentCount": "comment_cnt"})

    return df


def load_naver(issue):
    path = os.path.join(NAVER_PATH, f"{issue}_final_post_level.xlsx")
    if not os.path.exists(path):
        print(f"⚠️ 네이버 파일 없음: {path}")
        return pd.DataFrame()

    df = pd.read_excel(path)
    df = normalize_datetime_column(df)
    df = df.drop(columns=["키워드"], errors="ignore")

    df = df.rename(
        columns={
            "url": "content_id",
            "뉴스제목": "title_or_text"
        }
    )
    df["channel"] = "네이버"

    keep_cols = [
        "content_id", "channel", "언론사", "title_or_text",
        "post_sentiment", "post_s", "cmt_s_mean", "abs_sent_gap",
        "aligned_cnt_3h", "aligned_cnt_6h", "aligned_cnt_9h",
        "comment_cnt", "datetime","content"
    ]
    df = df[[c for c in keep_cols if c in df.columns]]

    return df


# ============================================================
# ✅ 4. 병합 및 저장
# ============================================================

os.makedirs(OUTPUT_PATH, exist_ok=True)

for issue in ISSUES:
    yt_df = load_youtube(issue)
    nv_df = load_naver(issue)

    combined = pd.concat([yt_df, nv_df], ignore_index=True)

    # ✅ comment_cnt NaN → 0
    if "comment_cnt" in combined.columns:
        combined["comment_cnt"] = pd.to_numeric(combined["comment_cnt"], errors="coerce").fillna(0).astype(int)

    # ✅ 날짜+시간 기준 정렬
    combined = combined.sort_values(by="datetime")

    # ✅ datetime 컬럼을 가장 왼쪽으로 이동
    cols = ["datetime"] + [c for c in combined.columns if c != "datetime"]
    combined = combined[cols]

    # 저장
    output_file = os.path.join(OUTPUT_PATH, f"{issue}_통합_final.xlsx")
    combined.to_excel(output_file, index=False, engine="openpyxl")

    print(f"✅ {issue} 완료 ({len(combined)}행 저장) → {output_file}")

print("\n🎯 모든 이슈 통합 완료 (시간 정보 유지 + datetime 맨 왼쪽 + comment_cnt=0 처리)")


✅ GS 완료 (208행 저장) → C:\Users\user\Desktop\PoC_v2\통합결과_날짜시간\GS_통합_final.xlsx
✅ KT 완료 (785행 저장) → C:\Users\user\Desktop\PoC_v2\통합결과_날짜시간\KT_통합_final.xlsx
✅ 롯데 완료 (840행 저장) → C:\Users\user\Desktop\PoC_v2\통합결과_날짜시간\롯데_통합_final.xlsx
✅ SKT 완료 (1375행 저장) → C:\Users\user\Desktop\PoC_v2\통합결과_날짜시간\SKT_통합_final.xlsx

🎯 모든 이슈 통합 완료 (시간 정보 유지 + datetime 맨 왼쪽 + comment_cnt=0 처리)


In [23]:
# import pandas as pd
# import os

# # ============================================================
# # ✅ 1. 기본 설정
# # ============================================================
# YOUTUBE_PATH = r"C:\Users\user\Desktop\PoC_v2\YouTube_결과\final_post"
# NAVER_PATH = r"C:\Users\user\Desktop\PoC_v2\공유폴더(딥테크)\final_post_1"
# OUTPUT_PATH = r"C:\Users\user\Desktop\PoC_v2\통합결과_날짜시간"

# ISSUES = ["GS", "KT", "롯데", "SKT"]

# # ============================================================
# # ✅ 2. 날짜 + 시간 통일 함수 (KST 유지)
# # ============================================================

# def normalize_datetime_column(df):
#     """YouTube: publishedAt / Naver: 게시일 → datetime (KST) 변환, 시간 포함"""
#     if "publishedAt" in df.columns:
#         df["datetime"] = (
#             pd.to_datetime(df["publishedAt"], errors="coerce", utc=True)
#             .dt.tz_convert("Asia/Seoul")
#             .dt.tz_localize(None)
#         )
#     elif "게시일" in df.columns:
#         df["datetime"] = (
#             pd.to_datetime(df["게시일"], errors="coerce", utc=True)
#             .dt.tz_convert("Asia/Seoul")
#             .dt.tz_localize(None)
#         )
#     else:
#         df["datetime"] = pd.NaT
#     return df


# # ============================================================
# # ✅ 3. 플랫폼별 로드 함수
# # ============================================================

# def load_youtube(issue):
#     path = os.path.join(YOUTUBE_PATH, f"{issue}_final_post_level.xlsx")
#     if not os.path.exists(path):
#         print(f"⚠️ 유튜브 파일 없음: {path}")
#         return pd.DataFrame()

#     df = pd.read_excel(path)
#     df = normalize_datetime_column(df)

#     df = df.rename(
#         columns={
#             "videoId": "content_id",
#             "gpt_post_sentiment": "post_sentiment",
#             "text_raw": "title_or_text"
#         }
#     )
#     df["channel"] = "유튜브"

#     keep_cols = [
#         "content_id", "channel", "channelTitle", "title_or_text",
#         "post_sentiment", "post_s", "cmt_s_mean", "abs_sent_gap",
#         "aligned_cnt_3h", "aligned_cnt_6h", "aligned_cnt_9h",
#         "viewCount", "likeCount", "commentCount", "subscriberCount", "datetime"
#     ]
#     df = df[[c for c in keep_cols if c in df.columns]]

#     # 유튜브 commentCount → comment_cnt 통일
#     if "commentCount" in df.columns:
#         df = df.rename(columns={"commentCount": "comment_cnt"})

#     return df


# def load_naver(issue):
#     path = os.path.join(NAVER_PATH, f"{issue}_final_post_level.xlsx")
#     if not os.path.exists(path):
#         print(f"⚠️ 네이버 파일 없음: {path}")
#         return pd.DataFrame()

#     df = pd.read_excel(path)
#     df = normalize_datetime_column(df)
#     df = df.drop(columns=["키워드"], errors="ignore")

#     df = df.rename(
#         columns={
#             "url": "content_id",
#             "뉴스제목": "title_or_text"
#         }
#     )
#     df["channel"] = "네이버"

#     keep_cols = [
#         "content_id", "channel", "언론사", "title_or_text",
#         "post_sentiment", "post_s", "cmt_s_mean", "abs_sent_gap",
#         "aligned_cnt_3h", "aligned_cnt_6h", "aligned_cnt_9h",
#         "comment_cnt", "datetime"
#     ]
#     df = df[[c for c in keep_cols if c in df.columns]]

#     return df


# # ============================================================
# # ✅ 4. 병합 및 저장
# # ============================================================

# os.makedirs(OUTPUT_PATH, exist_ok=True)

# for issue in ISSUES:
#     yt_df = load_youtube(issue)
#     nv_df = load_naver(issue)

#     combined = pd.concat([yt_df, nv_df], ignore_index=True)

#     # ✅ comment_cnt NaN → 0
#     if "comment_cnt" in combined.columns:
#         combined["comment_cnt"] = pd.to_numeric(combined["comment_cnt"], errors="coerce").fillna(0).astype(int)

#     # ✅ 날짜+시간 기준 정렬
#     combined = combined.sort_values(by="datetime")

#     # ✅ datetime 컬럼을 가장 왼쪽으로 이동
#     cols = ["datetime"] + [c for c in combined.columns if c != "datetime"]
#     combined = combined[cols]

#     # 저장
#     output_file = os.path.join(OUTPUT_PATH, f"{issue}_통합_final.xlsx")
#     combined.to_excel(output_file, index=False, engine="openpyxl")

#     print(f"✅ {issue} 완료 ({len(combined)}행 저장) → {output_file}")

# print("\n🎯 모든 이슈 통합 완료 (시간 정보 유지 + datetime 맨 왼쪽 + comment_cnt=0 처리)")


✅ GS 완료 (208행 저장) → C:\Users\user\Desktop\PoC_v2\통합결과_날짜시간\GS_통합_final.xlsx
✅ KT 완료 (785행 저장) → C:\Users\user\Desktop\PoC_v2\통합결과_날짜시간\KT_통합_final.xlsx
✅ 롯데 완료 (840행 저장) → C:\Users\user\Desktop\PoC_v2\통합결과_날짜시간\롯데_통합_final.xlsx
✅ SKT 완료 (1375행 저장) → C:\Users\user\Desktop\PoC_v2\통합결과_날짜시간\SKT_통합_final.xlsx

🎯 모든 이슈 통합 완료 (시간 정보 유지 + datetime 맨 왼쪽 + comment_cnt=0 처리)


In [7]:
df = pd.read_excel(r"C:\Users\user\Desktop\PoC_v2\통합결과_날짜시간\SKT_통합_final.xlsx")

df.head()

,datetime,content_id,channel,channelTitle,title_or_text,post_sentiment,post_s,cmt_s_mean,abs_sent_gap,aligned_cnt_3h,aligned_cnt_6h,aligned_cnt_9h,viewCount,likeCount,comment_cnt,subscriberCount,언론사,content
0,2025-04-22 15:29:35,f1N0C4HixGE,유튜브,TS 멤버십 클럽,SK텔레콤 유심 정보 유출? 예방법은? #sk텔레콤 #유심정보유출 #skt 안녕...,부정,-1,1.000000,2.00,0,0,0,2085.0,13.0,1,1970.0,NaN,NaN
1,2025-04-22 16:56:41,F6Qi4wMfzxI,유튜브,이데일리TV,"통신사 1위 ‘SK텔레콤’, 외부 해킹으로 고객 유심 정보 유출 (20250422)...",부정,-1,0.000000,1.00,0,0,0,1751.0,15.0,0,343000.0,NaN,NaN
2,2025-04-22 17:36:27,aFr3emetb9M,유튜브,cocojuan코코쥬안,"SK텔레콤 해킹 사고 유심 정보 유출, 유심보호서비스 가입 방법 SKT 유심 정보 ...",부정,-1,0.000000,1.00,0,0,0,7800.0,71.0,5,5600.0,NaN,NaN
3,2025-04-22 19:12:14,https://n.news.naver.com/mnews/article/028/000...,네이버,NaN,SKT “해킹 당했다…가입자 유심 정보 유출 의심” 자진신고,부정,-1,-0.538462,0.46,2,3,3,NaN,NaN,13,NaN,한겨레,https://n.news.naver.com/mnews/article/629/000...
4,2025-04-22 23:34:30,V71eooP07q4,유튜브,Cha Cha,SK텔레콤서해킹 피해발생 가입자 유심정보 유출 악성코드 공격받아#SKT#해킹#철저히...,부정,-1,0.000000,1.00,0,0,0,1361.0,8.0,0,62600.0,NaN,NaN


In [37]:
df.columns

Index(['datetime', 'content_id', 'channel', 'channelTitle', 'title_or_text',
       'post_sentiment', 'post_s', 'cmt_s_mean', 'abs_sent_gap',
       'aligned_cnt_3h', 'aligned_cnt_6h', 'aligned_cnt_9h', 'viewCount',
       'likeCount', 'comment_cnt', 'subscriberCount', '언론사'],
      dtype='object')

In [34]:
import pandas as pd
import os

# ------------------------------------------------------------
# 1️⃣ 경로 및 이슈 목록
# ------------------------------------------------------------
BASE_PATH = r"C:\Users\user\Desktop\PoC_v2\YouTube_결과\final_post"  # ✅ 유튜브 전용 경로
ISSUES = ["GS", "KT", "롯데", "SKT"]

summary = []

# ------------------------------------------------------------
# 2️⃣ 이슈별 유튜브 영상·댓글 집계
# ------------------------------------------------------------
for issue in ISSUES:
    file_path = os.path.join(BASE_PATH, f"{issue}_final_post_level.xlsx")

    if not os.path.exists(file_path):
        print(f"⚠️ {issue} 파일이 존재하지 않습니다: {file_path}")
        continue

    df = pd.read_excel(file_path)

    # 채널 컬럼 자동 탐색
    channel_col = None
    for col in df.columns:
        if "channel" in col.lower():
            channel_col = col
            break

    # -------------------------------
    # 🎯 집계 계산
    # -------------------------------
    num_channels = df[channel_col].nunique() if channel_col else None
    num_videos = df["videoId"].nunique() if "videoId" in df.columns else len(df)
    total_comments = df["commentCount"].sum() if "commentCount" in df.columns else 0
    avg_comments = round(total_comments / num_videos, 2) if num_videos > 0 else 0

    summary.append({
        "이슈": issue,
        "채널 수": num_channels,
        "유튜브 영상 수": num_videos,
        "유튜브 총 댓글 수": total_comments,
        "평균 댓글/영상": avg_comments
    })

# ------------------------------------------------------------
# 3️⃣ 결과 표 생성
# ------------------------------------------------------------
df_summary = pd.DataFrame(summary).sort_values("유튜브 총 댓글 수", ascending=False)
print("\n🎯 이슈별 유튜브 영상·댓글 요약표\n")
print(df_summary.to_markdown(index=False))

# ------------------------------------------------------------
# 4️⃣ 저장
# ------------------------------------------------------------
output_path = os.path.join(BASE_PATH, "이슈별_유튜브_전용_요약.xlsx")
df_summary.to_excel(output_path, index=False, engine="openpyxl")
print(f"\n✅ 저장 완료: {output_path}")



🎯 이슈별 유튜브 영상·댓글 요약표

| 이슈   |   채널 수 |   유튜브 영상 수 |   유튜브 총 댓글 수 |   평균 댓글/영상 |
|:-------|----------:|-----------------:|--------------------:|-----------------:|
| SKT    |       354 |              494 |               68249 |           138.16 |
| KT     |       204 |              349 |               28770 |            82.44 |
| 롯데   |       243 |              500 |               16194 |            32.39 |
| GS     |        17 |               22 |                 215 |             9.77 |

✅ 저장 완료: C:\Users\user\Desktop\PoC_v2\YouTube_결과\final_post\이슈별_유튜브_전용_요약.xlsx


In [ ]:
import pandas as pd
import os

# ------------------------------------------------------------
# 1️⃣ 경로 및 이슈 목록
# ------------------------------------------------------------
BASE_PATH = r"C:\Users\speec\OneDrive\Desktop\PoC_v2\공유폴더(딥테크)"
ISSUES = ["GS", "KT", "롯데", "SKT"]

# ------------------------------------------------------------
# 2️⃣ 댓글 수를 게시글에 추가하는 함수 + 전체 댓글 개수 출력
# ------------------------------------------------------------
def add_comment_count_to_posts(issue):
    """댓글 수를 게시글 파일에 추가한 뒤 저장하며, 전체 댓글 수도 출력"""
    # 파일 불러오기
    post_path = os.path.join(BASE_PATH, f"{issue}_뉴스감정분석_GPT_labeled.xlsx")
    cmt_path = os.path.join(BASE_PATH, f"{issue}_댓글감정분석_GPT_labeled.xlsx")

    posts = pd.read_excel(post_path)
    comments = pd.read_excel(cmt_path)

    # 병합 키 통일 (URL 기준)
    posts["videoId"] = posts["url"]
    comments["videoId"] = comments["url"]

    # ✅ 댓글 수 계산 (videoId 기준)
    comment_counts = comments.groupby("videoId").size().rename("comment_cnt").reset_index()

    # ✅ 게시글에 댓글수 추가 (없으면 0)
    posts = posts.merge(comment_counts, on="videoId", how="left").fillna({"comment_cnt": 0})
    posts["comment_cnt"] = posts["comment_cnt"].astype(int)

    # ✅ 전체 댓글 수 카운트
    total_comments = comments.shape[0]

    # ✅ 결과 저장
    output_path = os.path.join(BASE_PATH, f"{issue}_posts_with_comment_cnt.xlsx")
    posts.to_excel(output_path, index=False, engine="openpyxl")

    print(f"✅ {issue} 완료 — 게시글 {len(posts)}건, 댓글수 추가 후 저장됨")
    print(f"📂 저장 위치: {output_path}")
    print(f"💬 {issue} 이슈 댓글 총 {total_comments:,}개가 추가 및 집계되었습니다.\n")

# ------------------------------------------------------------
# 3️⃣ 실행
# ------------------------------------------------------------
for issue in ISSUES:
    add_comment_count_to_posts(issue)

print("\n🎯 모든 이슈 게시글에 댓글수가 추가되었습니다!")


✅ GS 완료 — 게시글 186건, 댓글수 추가 후 저장됨
📂 저장 위치: C:\Users\speec\OneDrive\Desktop\PoC_v2\공유폴더(딥테크)\GS_posts_with_comment_cnt.xlsx
💬 GS 이슈 댓글 총 593개가 추가 및 집계되었습니다.

✅ KT 완료 — 게시글 436건, 댓글수 추가 후 저장됨
📂 저장 위치: C:\Users\speec\OneDrive\Desktop\PoC_v2\공유폴더(딥테크)\KT_posts_with_comment_cnt.xlsx
💬 KT 이슈 댓글 총 2,248개가 추가 및 집계되었습니다.

✅ 롯데 완료 — 게시글 340건, 댓글수 추가 후 저장됨
📂 저장 위치: C:\Users\speec\OneDrive\Desktop\PoC_v2\공유폴더(딥테크)\롯데_posts_with_comment_cnt.xlsx
💬 롯데 이슈 댓글 총 2,287개가 추가 및 집계되었습니다.

✅ SKT 완료 — 게시글 881건, 댓글수 추가 후 저장됨
📂 저장 위치: C:\Users\speec\OneDrive\Desktop\PoC_v2\공유폴더(딥테크)\SKT_posts_with_comment_cnt.xlsx
💬 SKT 이슈 댓글 총 15,388개가 추가 및 집계되었습니다.


🎯 모든 이슈 게시글에 댓글수가 추가되었습니다!


In [ ]:
import pandas as pd
import os

# ============================================================
# ✅ 1. 기본 설정
# ============================================================
BASE_PATH = r"C:\Users\user\Desktop\PoC_v2\공유폴더(딥테크)"
ISSUES = ["GS", "KT", "롯데", "SKT"]

# ============================================================
# ✅ 2. 함수 정의
# ============================================================

def load_data(issue):
    """posts / comments 데이터 로드 + 날짜 처리"""
    posts = pd.read_excel(os.path.join(BASE_PATH, f"{issue}_posts_with_comment_cnt.xlsx"))
    comments = pd.read_excel(os.path.join(BASE_PATH, f"{issue}_댓글감정분석_GPT_labeled.xlsx"))
    
    posts["publishedAt"] = pd.to_datetime(posts["게시일"], errors="coerce", utc=True)
    comments["publishedAt"] = pd.to_datetime(comments["post_date"], errors="coerce", utc=True)
    
    return posts, comments


def merge_post_comment(posts, comments):
    """게시글과 댓글 데이터 병합 + 시간차 계산"""
    merged = pd.merge(
        posts,
        comments[["url", "publishedAt", "gpt_sentiment"]],
        on="url",
        how="left",
        suffixes=("_post", "_comment")
    )

    merged["hours_after"] = (
        (merged["publishedAt_comment"] - merged["publishedAt_post"]).dt.total_seconds() / 3600
    )
    return merged


def calc_alignment(merged):
    """게시글-댓글 감정 정렬도 계산"""
    def to_score(x):
        if isinstance(x, str):
            if "부정" in x:
                return -1
            elif "긍정" in x:
                return 1
        return 0

    merged["post_s"] = merged["gpt_sentiment_post"].apply(to_score)
    merged["cmt_s"] = merged["gpt_sentiment_comment"].apply(to_score)

    # videoId 컬럼이 없으므로 url 기준으로 그룹화
    grouped = (
        merged.groupby("url")
        .agg(
            키워드=("키워드", "first"),
            언론사=("언론사", "first"),
            뉴스제목=("뉴스제목", "first"),
            게시일=("게시일", "first"),
            post_sentiment=("gpt_sentiment_post", "first"),
            post_s=("post_s", "first"),
            cmt_s_mean=("cmt_s", "mean"),
            comment_cnt=("comment_cnt", "first"),
        )
        .reset_index()
    )

    # 감정 차이
    grouped["abs_sent_gap"] = (grouped["post_s"] - grouped["cmt_s_mean"]).abs().round(2)

    # 3/6/9시간 내 동일감정 댓글 수 카운트
    for h in [3, 6, 9]:
        cond = (merged["hours_after"] <= h) & (merged["post_s"] == merged["cmt_s"])
        aligned = merged[cond].groupby("url").size()
        grouped[f"aligned_cnt_{h}h"] = grouped["url"].map(aligned).fillna(0).astype(int)

    return grouped


# ============================================================
# ✅ 3. 실행 루프
# ============================================================
for issue in ISSUES:
    print(f"\n🚀 {issue} 처리 중...")

    posts, comments = load_data(issue)
    merged = merge_post_comment(posts, comments)
    result = calc_alignment(merged)

    # Excel 저장 전 timezone 제거
    for col in merged.select_dtypes(include=["datetimetz"]).columns:
        merged[col] = merged[col].dt.tz_localize(None)
    for col in result.select_dtypes(include=["datetimetz"]).columns:
        result[col] = result[col].dt.tz_localize(None)

    # 저장
    merged.to_excel(os.path.join(BASE_PATH, f"{issue}_merged_timegap.xlsx"), index=False, engine="openpyxl")
    result.to_excel(os.path.join(BASE_PATH, f"{issue}_final_post_level.xlsx"), index=False, engine="openpyxl")

    
print("\n🎯 모든 이슈 처리 완료!")



🚀 GS 처리 중...

🚀 KT 처리 중...

🚀 롯데 처리 중...

🚀 SKT 처리 중...

🎯 모든 이슈 처리 완료!


In [39]:
import pandas as pd
import os

# ------------------------------------------------------------
# 1️⃣ 경로 설정
# ------------------------------------------------------------
BASE_PATH = r"C:\Users\user\Desktop\PoC_v2\통합결과_날짜시간"
ADD_PATH = r"C:\Users\user\Desktop\PoC_v2\데이터"
OUTPUT_PATH = r"C:\Users\user\Desktop\PoC_v2\통합결과_추가채널"

os.makedirs(OUTPUT_PATH, exist_ok=True)

ISSUES = ["GS", "KT", "롯데", "SKT"]

# ------------------------------------------------------------
# 2️⃣ 이슈별 병합 루프
# ------------------------------------------------------------
for issue in ISSUES:
    base_file = os.path.join(BASE_PATH, f"{issue}_통합_final.xlsx")
    add_file = os.path.join(ADD_PATH, f"merged_output_{issue.lower()}.xlsx")
    output_file = os.path.join(OUTPUT_PATH, f"{issue}_통합_final_추가채널.xlsx")

    if not os.path.exists(base_file):
        print(f"⚠️ {issue}: 베이스 파일 없음 → {base_file}")
        continue
    if not os.path.exists(add_file):
        print(f"⚠️ {issue}: 추가 파일 없음 → {add_file}")
        continue

    # --------------------------------------------------------
    # 3️⃣ 데이터 불러오기
    # --------------------------------------------------------
    base_df = pd.read_excel(base_file)
    add_df = pd.read_excel(add_file)

    # --------------------------------------------------------
    # 4️⃣ 컬럼명 통일
    # --------------------------------------------------------
    add_df = add_df.rename(columns={
        "date": "datetime",
        "title": "channelTitle",
        "text": "title_or_text"
    })

    # --------------------------------------------------------
    # 5️⃣ 날짜 변환
    # --------------------------------------------------------
    base_df["datetime"] = pd.to_datetime(base_df["datetime"], errors="coerce")
    add_df["datetime"] = pd.to_datetime(add_df["datetime"], errors="coerce")

    # --------------------------------------------------------
    # 6️⃣ 병합 (outer join)
    # --------------------------------------------------------
    merged = pd.merge(
        base_df,
        add_df[["datetime", "channel", "channelTitle", "title_or_text"]],
        on="datetime",
        how="outer",
        suffixes=("_base", "_add")
    )

    # --------------------------------------------------------
    # 7️⃣ 저장
    # --------------------------------------------------------
    merged.to_excel(output_file, index=False, engine="openpyxl")

    print(f"✅ {issue} 병합 완료 — {len(merged):,}행 저장")
    print(f"📁 {output_file}")

print("\n🎯 모든 이슈 통합 및 추가채널 병합 완료!")


✅ GS 병합 완료 — 292행 저장
📁 C:\Users\user\Desktop\PoC_v2\통합결과_추가채널\GS_통합_final_추가채널.xlsx
✅ KT 병합 완료 — 934행 저장
📁 C:\Users\user\Desktop\PoC_v2\통합결과_추가채널\KT_통합_final_추가채널.xlsx
✅ 롯데 병합 완료 — 1,181행 저장
📁 C:\Users\user\Desktop\PoC_v2\통합결과_추가채널\롯데_통합_final_추가채널.xlsx
✅ SKT 병합 완료 — 2,575행 저장
📁 C:\Users\user\Desktop\PoC_v2\통합결과_추가채널\SKT_통합_final_추가채널.xlsx

🎯 모든 이슈 통합 및 추가채널 병합 완료!


In [ ]:
import pandas as pd
import os

# ------------------------------------------------------------
# 1️⃣ 경로 및 설정
# ------------------------------------------------------------
BASE_PATH = r"C:\Users\user\Desktop\PoC_v2\통합결과_추가채널"
OUTPUT_PATH = r"C:\Users\user\Desktop\PoC_v2\통합결과_정리완료"

os.makedirs(OUTPUT_PATH, exist_ok=True)

ISSUES = ["GS", "KT", "롯데", "SKT"]

# ------------------------------------------------------------
# 2️⃣ 루프 시작
# ------------------------------------------------------------
for issue in ISSUES:
    file_path = os.path.join(BASE_PATH, f"{issue}_통합_final_추가채널.xlsx")

    if not os.path.exists(file_path):
        print(f"⚠️ {issue} 파일이 존재하지 않음: {file_path}")
        continue

    df = pd.read_excel(file_path)

    # --------------------------------------------------------
    # 3️⃣ 컬럼 존재 여부 확인
    # --------------------------------------------------------
    cols = df.columns.tolist()
    print(f"\n📂 {issue} 컬럼: {cols}")

    # --------------------------------------------------------
    # 4️⃣ 채널 컬럼 통일
    # --------------------------------------------------------
    if "channel_base" in df.columns and "channel_add" in df.columns:
        df["channel"] = df["channel_add"].combine_first(df["channel_base"])
    elif "channel_add" in df.columns:
        df["channel"] = df["channel_add"]
    elif "channel_base" in df.columns:
        df["channel"] = df["channel_base"]

    # --------------------------------------------------------
    # 5️⃣ 제목·본문 컬럼 통합
    # --------------------------------------------------------
    title_cols = [c for c in df.columns if "title_or_text" in c or "channelTitle" in c]
    df["title_or_text"] = df.get("title_or_text_add", "").astype(str).fillna("") + " " + \
                          df.get("channelTitle_add", "").astype(str).fillna("")
    # 만약 title_or_text_base가 있다면 우선 사용하고, 비어있으면 통합본 사용
    if "title_or_text_base" in df.columns:
        df["title_or_text"] = df["title_or_text_base"].combine_first(df["title_or_text"])

    # --------------------------------------------------------
    # 6️⃣ 불필요한 컬럼 제거 (_base, _add)
    # --------------------------------------------------------
    df = df[[c for c in df.columns if not (c.endswith("_base") or c.endswith("_add"))]]

    # --------------------------------------------------------
    # 7️⃣ 정리 및 저장
    # --------------------------------------------------------
    output_file = os.path.join(OUTPUT_PATH, f"{issue}_통합_final_정리완료.xlsx")
    df.to_excel(output_file, index=False, engine="openpyxl")

    print(f"✅ {issue} 정리 완료 ({len(df):,}행 저장)")
    print(f"📁 저장 위치: {output_file}")

print("\n🎯 모든 이슈 정리 완료!")


+트위터와 블로그 감정수행을 안해서 최종본에 감정분석 추가

In [2]:
!pip install openai tqdm pandas --quiet


In [5]:
# ============================================================
# ✅ 블로그·트위터 전용 GPT 감정분석 (PoC 통합데이터 4이슈 자동)
# ============================================================

import pandas as pd
import json, time, re, os
from tqdm import tqdm
from openai import OpenAI

# 🔑 OpenAI API 키
import os
api_key = os.getenv("OPENAI_API_KEY", "YOUR_API_KEY_HERE")
client = OpenAI(api_key=api_key)
# ------------------------------------------------------------
# 1️⃣ GPT 감정분석 함수 (배치 + 캐싱)
# ------------------------------------------------------------
cache = {}

def gpt_batch_sentiment(texts, batch_size=20, sleep_sec=0.5):
    """텍스트 리스트를 GPT-4o-mini로 배치 감정분석"""
    results = []

    for i in tqdm(range(0, len(texts), batch_size), desc="GPT 감정분석 중"):
        batch = texts[i:i + batch_size]
        new_texts = [t for t in batch if t not in cache]

        # ✅ 캐시에 모두 있으면 바로 append
        if not new_texts:
            results.extend([cache[t] for t in batch])
            continue

        # ✅ 프롬프트 생성
        batch_prompt = "\n".join([f"{j+1}. {t}" for j, t in enumerate(new_texts, start=1)])
        prompt = f"""
        다음 문장들의 감정을 JSON 배열로 출력해.
        각 문장은 반드시 'sentiment'(부정/중립/긍정)와 'confidence'(0~1)를 포함해야 해.

        예시:
        [
          {{"id":1, "sentiment":"부정", "confidence":0.92}},
          {{"id":2, "sentiment":"중립", "confidence":0.53}}
        ]

        문장:
        {batch_prompt}
        """

        try:
            # ✅ GPT 호출
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "너는 한국어 감정분석 전문가야."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.2,
            )

            output = response.choices[0].message.content.strip()
            match = re.search(r"\[.*\]", output, re.S)
            parsed = json.loads(match.group(0)) if match else []

            # ✅ 결과 정리 및 캐싱
            for j, t in enumerate(new_texts, start=1):
                if j <= len(parsed):
                    s = parsed[j-1].get("sentiment", "중립")
                    c = parsed[j-1].get("confidence", 0.5)
                else:
                    s, c = "중립", 0.0
                cache[t] = {"sentiment": s, "confidence": c}

            results.extend([cache[t] for t in batch])

        except Exception as e:
            print(f"❌ 오류 발생: {e}")
            results.extend([{"sentiment": "중립", "confidence": 0.0} for _ in batch])

        time.sleep(sleep_sec)

    return results


# ------------------------------------------------------------
# 2️⃣ 경로 및 이슈 목록
# ------------------------------------------------------------
INPUT_DIR = r"C:\Users\user\Desktop\PoC_v2\통합결과_정리완료"
OUTPUT_DIR = r"C:\Users\user\Desktop\PoC_v2\통합결과_추가채널"
ISSUES = ["GS", "KT", "롯데", "SKT"]

os.makedirs(OUTPUT_DIR, exist_ok=True)

# ------------------------------------------------------------
# 3️⃣ 이슈별 감정분석 실행 루프
# ------------------------------------------------------------
for issue in ISSUES:
    print(f"\n🚀 {issue} 감정분석 시작 중...")

    input_path = fr"{INPUT_DIR}\{issue}_통합_final_정리완료.xlsx"
    output_path = fr"{OUTPUT_DIR}\{issue}_통합_final_추가채널_감정분석완료.xlsx"

    # ✅ 파일 유효성 확인
    if not os.path.exists(input_path):
        print(f"⚠️ {issue} 파일이 존재하지 않습니다 → {input_path}")
        continue

    # ✅ 데이터 불러오기
    df = pd.read_excel(input_path)
    print(f"📂 불러온 데이터: {len(df):,}행")

    # ✅ blog / twitter 대상 필터링
    if "channel" not in df.columns or "title_or_text" not in df.columns:
        print(f"⚠️ {issue} 파일에 필수 컬럼이 없습니다 (channel, title_or_text)")
        continue

    mask = df["channel"].astype(str).str.lower().isin(["blog", "twitter"])
    target_df = df[mask].copy()

    print(f"🧩 감정분석 대상: {len(target_df):,}건 (blog/twitter)")

    # ✅ 감정분석 실행
    if len(target_df) == 0:
        print(f"⚠️ {issue}에는 blog/twitter 데이터가 없습니다. 건너뜀.")
        continue

    texts = target_df["title_or_text"].astype(str).fillna("").str.strip().str[:300].tolist()
    results = gpt_batch_sentiment(texts, batch_size=20)

    target_df["post_sentiment"] = [r["sentiment"] for r in results]
    target_df["gpt_confidence"] = [r["confidence"] for r in results]

    # ✅ 원본 병합 (해당 인덱스만 업데이트)
    df.loc[target_df.index, "post_sentiment"] = target_df["post_sentiment"]
    df.loc[target_df.index, "gpt_confidence"] = target_df["gpt_confidence"]

    # ✅ 결과 저장
    df.to_excel(output_path, index=False, engine="openpyxl")

    print(f"✅ {issue} 감정분석 완료!")
    print(f"📁 저장 경로: {output_path}")
    print(f"📊 blog/twitter 감정분석 완료 행 수: {len(target_df):,}")

print("\n🎯 모든 이슈 감정분석 완료!")



🚀 GS 감정분석 시작 중...
📂 불러온 데이터: 292행
🧩 감정분석 대상: 84건 (blog/twitter)


GPT 감정분석 중: 100%|██████████| 5/5 [00:43<00:00,  8.62s/it]


✅ GS 감정분석 완료!
📁 저장 경로: C:\Users\user\Desktop\PoC_v2\통합결과_추가채널\GS_통합_final_추가채널_감정분석완료.xlsx
📊 blog/twitter 감정분석 완료 행 수: 84

🚀 KT 감정분석 시작 중...
📂 불러온 데이터: 934행
🧩 감정분석 대상: 149건 (blog/twitter)


GPT 감정분석 중: 100%|██████████| 8/8 [01:03<00:00,  7.97s/it]


✅ KT 감정분석 완료!
📁 저장 경로: C:\Users\user\Desktop\PoC_v2\통합결과_추가채널\KT_통합_final_추가채널_감정분석완료.xlsx
📊 blog/twitter 감정분석 완료 행 수: 149

🚀 롯데 감정분석 시작 중...
📂 불러온 데이터: 1,181행
🧩 감정분석 대상: 342건 (blog/twitter)


GPT 감정분석 중: 100%|██████████| 18/18 [02:21<00:00,  7.86s/it]


✅ 롯데 감정분석 완료!
📁 저장 경로: C:\Users\user\Desktop\PoC_v2\통합결과_추가채널\롯데_통합_final_추가채널_감정분석완료.xlsx
📊 blog/twitter 감정분석 완료 행 수: 342

🚀 SKT 감정분석 시작 중...
📂 불러온 데이터: 2,575행
🧩 감정분석 대상: 1,202건 (blog/twitter)


GPT 감정분석 중: 100%|██████████| 61/61 [08:44<00:00,  8.60s/it]


✅ SKT 감정분석 완료!
📁 저장 경로: C:\Users\user\Desktop\PoC_v2\통합결과_추가채널\SKT_통합_final_추가채널_감정분석완료.xlsx
📊 blog/twitter 감정분석 완료 행 수: 1,202

🎯 모든 이슈 감정분석 완료!
